# Handle Conflicts

## Set monitoring alias

In [ ]:
#### db-2 ####
alias pg_sub_stats='echo "SELECT * FROM pg_stat_subscription_stats; \watch i=5 c=4" | psql' # Cumulative
alias pg_sub='psql -c "SELECT * FROM pg_stat_subscription;"' # Dynamic

## Create Conflict

In [ ]:
#### db-2 ####
psql -c "ALTER TABLE replicated_a ADD CONSTRAINT a_unique_data UNIQUE(data);"

#### db-1 ####
psql -c "INSERT INTO replicated_a VALUES (2, 'z'), (3, 'z');"

#### db-2 ####
## Check out the subscription
pg_select_rep_a
pg_logs -n 100
pg_sub
pg_sub_stats


#### db-1 ####
## Look on the delta
psql << EOM
SELECT 
    pg_size_pretty(pg_current_wal_lsn() - confirmed_flush_lsn) fall_behaind_size
FROM pg_replication_slots
WHERE slot_name = 'sub_db1_all_tbl';
EOM

## Disable Subscription - Quit Loop

In [ ]:
#### db-2 ####
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl SET (DISABLE_ON_ERROR='on');"

## Check out the subscription
pg_sub
pg_sub_stats

## Handle By Changing Constraint

In [ ]:
#### db-2 ####
## Remove constraint to handle error
psql -c "ALTER TABLE replicated_a DROP CONSTRAINT a_unique_data;"

## Enable the subscription
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl ENABLE;"
pg_select_rep_a

## Handle By Skipping LSN

In [ ]:
###### Create conflict on replicated_b ######
#### db-2 ####
psql -c "ALTER TABLE replicated_b ADD CONSTRAINT b_unique_data UNIQUE(data);"

#### db-1 ####
psql -c "INSERT INTO replicated_b VALUES (2, 'z'), (3, 'z');"
psql -c "INSERT INTO replicated_b VALUES (4, 'z'), (5, 'x');"


###### Handle by skipping LSN ######
## Find problematic LSN
pg_logs

## Skip LSN
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl SKIP (LSN='<lsn>');"

## Enable the subscription
psql -c "ALTER SUBSCRIPTION sub_db1_all_tbl ENABLE;"
pg_select_rep_b
pg_sub
pg_sub_stats